# Implementing XGBoost using Grid Search as well as Hyperopt
- modeltype: **XGBoost** 
- train: **0.886**
- test: **0.865**

## Flow of the notebook
* 0. Rationale behind choosing XGBoost
* 1. Reading Data and importing libraries
* 2. Data Manipulation
* 3. Implementing XGBoost

## 0. Rationale behind choosing XGBoost

There are many articles online that has compared XGBoost, CatBoost and LightGBM and in most of them, XGBoost had a relatively lesser efficiency, and in few of them, it also had a relatively lesser accuracy.

But after researching a bit, XGBoost seemed a highly popular algorithm in data scienstist community and also several top performing kagglers had mentioned this algorithm in the past. So even though being a bit older and relatively inefficient compared to CatBoost and LighGBM I was curious to get my hands on it.

I have explained the working and parameter tuning of the algorithm in the section 3.

## 1. Reading Data & Importing Libraries

In [1]:
# Basic Data operations
import os
import numpy as np
import pandas as pd
import pickle


# H20 and Grid Search
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch


# Hyper Opt
import hyperopt
from numpy.random import RandomState
from sklearn.metrics import roc_auc_score

# Uncomment this if the category_encoder version is less than 2.1.0
# ! pip install category_encoders # This will update Sk-learn as well
from category_encoders import *

In [2]:
# Set directories
print(os.getcwd())
dirRawData = "../input/"
dirPData   = "../PData/"
dirPOutput = "../POutput/"

/home/jovyan/Final_Project/Pcode


In [3]:
f_name = dirPData + '01_df_250k.pickle'

with (open(f_name, "rb")) as f:
    dict_ = pickle.load(f)

df_train = dict_['df_train']
df_test  = dict_['df_test']

del f_name, dict_

In [4]:
f_name = dirPData + '01_vars.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)

vars_ind_numeric     = dict_['vars_ind_numeric']
vars_ind_hccv        = dict_['vars_ind_hccv']
vars_ind_categorical = dict_['vars_ind_categorical']
vars_notToUse        = ['unique_id']
var_dep              = dict_['var_dep']

del f_name, dict_

## 2. Data manipulation

### 2.2 Treating NAs

* **Handling NAs**

One of the advantage of XGBoost over other Gradient Boosting algorithm is it efficiently handles the missing values. As Tianqi Chen has discussed in his paper
> When a value is missing in the sparse matrix x, the instance is classified into the default direction. There are two choices
of default direction in each branch. The optimal default directions are learnt from the data. The key improvement is to only visit the non-missing entries. The presented algorithm treats the non-presence as a missing value and learns the best direction to handle missing values. 

And as mentioned in the H20 documentation: 
>  Missing values are interpreted as containing information (i.e., missing for a reason), rather than missing at random. During tree building, split decisions for every node are found by minimizing the loss function and treating missing values as a separate category that can go either left or right. XGBoost will automatically learn which is the best direction to go when a value is missing.

Following this, we have decided not to do any operations on missing value (except for deleting 'c02' column becuase it contain >50% of missing values in test data). 

-99 values will be treated in the H20 Frame, right now we are only removing 'C02' variable.

In [5]:


# Since C02 has a lot of NAs (>50%), we will drop it from both train and test
df_test.drop('c02', axis=1, inplace = True)
df_train.drop('c02', axis=1, inplace = True)
vars_ind_categorical.remove('c02')


### 2.3 Cardinality
   For treating the hccv we have used Target Encoders (Sk-Learn). We have also observed that there is oversampling of few factors 
    while others are under-sampled therefore we have used smoothing factor = 4. Smoothing of 4 is chosen by following few online 
    blogs. Also scikit learn is used to do the target encoding because I was a bit confused with H20 target encoding. Additionally,
    H20 automatically perform one_hot encoding on categorical variables so they have not been treated.


Note: I needed to update the scikit-learn and category_ecoders library

In [8]:
# To see the distribution of factors in a hcc variable
# df_train[vars_ind_hccv].nunique()
# df_train['e18'].value_counts()

In [6]:
# Target encoders on Train
enc = TargetEncoder(cols=vars_ind_hccv, smoothing =4)
enc.fit_transform(df_train, df_train['target'])
df_train = enc.transform(df_train, df_train['target'])

In [7]:
# Target encoders on Test
df_test['target'] = np.nan # Creating dummy 'target' variable for using the the enc.transform function
df_test = enc.transform(df_test) # applying the already trained encoder
df_test.drop(columns=['target'], inplace=True) # Dropping dummy 'target' variable

Running H20 cluster

In [11]:
# h2o.cluster().shutdown()

AttributeError: 'NoneType' object has no attribute 'shutdown'

In [8]:
h2o.init(port=54321, max_mem_size = "14g") # Asking h20 to use 14 GB of Ram
h2o.connect()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_212"; OpenJDK Runtime Environment (build 1.8.0_212-8u212-b03-0ubuntu1.18.04.1-b03); OpenJDK 64-Bit Server VM (build 25.212-b03, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp17mtc_0v
  JVM stdout: /tmp/tmp17mtc_0v/h2o_jovyan_started_from_python.out
  JVM stderr: /tmp/tmp17mtc_0v/h2o_jovyan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,"1 year, 2 months and 8 days !!!"
H2O cluster name:,H2O_from_python_jovyan_s0rtq1
H2O cluster total nodes:,1
H2O cluster free memory:,12.44 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Connecting to H2O server at http://localhost:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,"1 year, 2 months and 8 days !!!"
H2O cluster name:,H2O_from_python_jovyan_s0rtq1
H2O cluster total nodes:,1
H2O cluster free memory:,12.44 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


<H2OConnection to http://localhost:54321, no session>

We have removed 'unique_id' from Frames as it was not useful in prediction


In [9]:
vars_to_use = vars_ind_numeric + vars_ind_categorical
vars_to_use.remove('unique_id')
vars_ind_numeric.remove('unique_id')


h2o_df_train = h2o.H2OFrame(df_train[[var for var in vars_to_use+var_dep ]], destination_frame = 'df_train') # Train Frame
h2o_df_test  = h2o.H2OFrame(df_test[[var for var in vars_to_use]], destination_frame = 'df_test') # Test Frame

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Converting -99 to NAs. We have not done this before becuase as discussed earlier, converting pandas NAs to H20 Frame were not consistent and was giving an error (may be due to the older version of H20 in the image).

In [10]:
# Converting -99 to NA in train
for var in vars_ind_numeric:
    h2o_df_train[h2o_df_train[var] == -99.0 , var] = None
    
# Converting -99 to NA in test
for var in vars_ind_numeric:
    h2o_df_test[h2o_df_test[var] == -99.0 , var] = None

In [11]:
# H20 Document suggest to make dependant variable as factor for classiication task
h2o_df_train[var_dep] = h2o_df_train[var_dep].asfactor()

### 2.4 Interactions
Documentation regarding the interaction in XGBoost was not clear, therefore after  searching for online, especially [here](https://www.kaggle.com/c/bosch-production-line-performance/discussion/24418) making interaction seemed to be a logical choice given it has drastically improved the past model (GLM) performance.

I have not used the interaction on all variables, instead I have used it only on 2 variables ('f03' and 'e11'). These two 
variables have been chosen after running GLM model and then calculating variables importance.
I did not use all categorical variables, as it could not improve the performance significantly but made the
model quite complex to comprehend. Therefore, it was a trade-off between accuracy and complexity.


In [44]:
# Code for runing GLM to see which all variables are important

# Idea has been taken from: https://aichamp.wordpress.com/2017/09/29/python-example-of-building-glm-gbm-and-random-forest-
# binomial-model-with-h2o/


# from h2o.estimators.glm import H2OGeneralizedLinearEstimator
# glm_logistic = H2OGeneralizedLinearEstimator(family = "binomial")
# glm_logistic.train(x=vars_to_use , y= 'target', training_frame=h2o_df_train, model_id="glm_logistic")
# preds = glm_logistic.predict(h2o_df_test)
# df_test['Predicted'] = np.round(preds[2].as_data_frame(), 5)
# df_preds_dt = df_test[['unique_id', 'Predicted']].copy()
# df_test[['unique_id', 'Predicted']].to_csv(dirPOutput + '1st.csv', index=False)
# log_var_imp = glm_logistic.varimp(use_pandas=True).head()
# log_var_imp.loc[0:5, 'variable'].tolist()

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e20' has levels not trained on: [30146, BE271, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e13' has levels not trained on: [Q, S]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e24' has levels not trained on: [J, M, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e03' has levels not trained on: [J, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'c09' has levels not trained on: [nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'a18' has levels not trained on: [D]
  warnings.warn(w)


['f10', 'f03.F', 'e19', 'e11.A', 'f03.E']

I have chosen min_occurence as int(len(h2o_df_train)/40) after many trial and error.

With int(len(h2o_df_train)/40) on 250K train data I was getting 5 factors for each variable. I believe performing 
interactions on top 4-5 variables rather than all the variables having occurrence > 10/20 would make the model
faster and more interpretable

In [12]:
# Train Frame
interaction_frame_train = h2o_df_train.interaction(['f03', 'e11'], pairwise = False, max_factors = 100,
                                                   min_occurrence = int(len(h2o_df_train)/40))

# Test Frame
interaction_frame_test = h2o_df_test.interaction(['f03', 'e11'], pairwise = False, max_factors = 100, 
                                                 min_occurrence = int(len(h2o_df_train)/40))

# Cbinding interaction frame to train and test
h2o_df_train = h2o_df_train.cbind(interaction_frame_train)
h2o_df_test = h2o_df_test.cbind(interaction_frame_test)

Interactions progress: |██████████████████████████████████████████████████| 100%
Interactions progress: |██████████████████████████████████████████████████| 100%


In [13]:
# incuding interaction frame's variable to variables list
vars_to_use = vars_to_use + ['f03_e11']
vars_ind_numeric = vars_ind_numeric + ['f03_e11']

## 3. Implementing XGBoost
* **My understanding of the algorithm**

XGBoost (Extreme Gradient Boosting) is a boosting algorithm. Boosting refers to ensemble technique where several models (tree models) are being used to make the final model.

Boosting models are build sequentially by minimising the errors from the previous model and Gradient Boosting is an advance version of boosting where the algorithm employs gradient descent method to minimise the error. The main difference between Bagging and Boosting is, Bagging combines different models while Boosting use the previous model to make a new model with less errors.

* **One_hot encoding**

Since XGBoost is dependent on Tree based algorithm, it was safe to assume that XGBoost do not require one_hot encoding. But after going through the paper [Chen T.,Guestrin C., XGBoost: A Scalable Tree Boosting System](https://arxiv.org/pdf/1603.02754.pdf) I realised that one_hot encoding is important if there are categorical variables to make XGBoost perfom better. 

After Going through H20 documentation, I realised that the algorithm, similar to GLM, automatically performs one_hot encoding for XGBoost and thus we do not have to manually do that.

* **Spline**

Since XGBoost is based on Tree based algorithm, we have not used splines.

### 3.1 **Grid Search Vs HyperOpt**

For my own understanding, I tried both, Grid Search and HyperOpt.

Grid search processed only 2 algorithm in 2 hours optimising 5 hyper-parameters with 5 nfolds. It's Train AUC was = 0.889 and valid AUC = 0.879. Prediction accuracy on Kaggle for Grid Search algorithm was 0.865.

On the other hand, HyperOpt explored 10 algorithms in 5 hours minutes (limit of 30 mins per algorithm) and returned train AUC of 0.886, validation AUC of 0.878 and Kaggle Test data AUC of 0.865. Even though the accuracy was not actually significant, I still decided to go for Hyperopt to get my hands on it.

### 3.2 Tuning Hyper-Parameters
**Hyper-paramters to optimize**
* Max_depth: It sets the number of maximum tree depths. Increasing the depth makes the model complex and may lead to overfitting. Default value is 6. By going through few tutorials, I have kept the values from 4 to 8.
* sample_rate: sample_rate tell XGBoost how much data sample should be collected to grow the tree. default is 1, which means 100% data is to be used. Higher values may increase the train accuracy, but for the accuracy on test data, sampling is suggested [Friedman, 1999](https://statweb.stanford.edu/~jhf/ftp/stobst.pdf). I have the values of 0.4, 0.6, 0.8 and 0.9
* min_rows: Specify the minimum number of observations for a leaf. Default value is 1, but in our model, value 1 may make the algorithm time consuming and might not significantly improve the accuracy. Therefore, I have chosen value of 10,20,30 and 40
* ntrees: It tell XGBoost the numbers of tree to build. I have tried 200 trees and observed that there is a need to build more trees to get it converged. Also, since we have taken small learn_rate values, we would require more tree. Therefore, I have set the values to 1000, 2000, 3000 and 4000.
* learn_rate: As per my understanding, learn_rate here is similar to learning-rate in gradient boosting. I couldn't get much information from the H20 documentation. Therefore, I reviewed certain online blogs to see how others have comprehended and used this value. After researching for a while I selected 0.01, 0.005 and 0.001 because others have used these values in line to the number of trees we are using.

**Fixed Hyper-parameters** have been discussed in the code.

### Grid Search

In [87]:
max_depth = [4, 5, 6, 7, 8]
sample_rate = [0.4, 0.6, 0.8, 0.9]
min_rows = [10, 20, 30, 40] # 
ntrees = [1000, 2000, 3000, 4000]
learn_rate = [0.01, 0.005, 0.001]

search_crit = {'strategy': "RandomDiscrete",#  The default strategy, "Cartesian", covers the entire space of h-p 
               # combinations but takes too much time. Therefore we have used "Random Discrete Strategy"
               "max_runtime_secs": 7200, #  maximum time allowed for he algorithm to run
               "max_models": 10, # maximum models to return
               "seed": 2020,  
               "stopping_rounds" : 4, # Stops training when the option selected for stopping_metric doesn’t improve for
               # the specified number of training rounds, based on a simple moving average. 
               "stopping_metric" : "AUC",# Since the kaggle score is based upon AUC, therefore I am using AUC as stopping metric
               "stopping_tolerance" : 1e-3} # to stop training if the improvement is less than this value. 1e-3 is the default
                # value

hyper_params = {
      "ntrees" : ntrees 
    , "max_depth" : max_depth 
    , "learn_rate" : learn_rate
    , "sample_rate" : sample_rate
    , "min_rows" : min_rows 
}
XGB_grid = H2OGridSearch(H2OXGBoostEstimator(col_sample_rate_per_tree = .9, # Specify the column subsampling rate per tree. It
                                             # is multiplicative with col_sample_rate, so setting both parameters to 0.8,
                                             # for example, results in 64% of columns being considered at any given node to 
                                             # split.0.9 has been taken because it seemed the most common choice among the 
                                             # online blogs
                                             score_tree_interval = 100, # Scoring the model after every so many trees
                                             nfolds = 5 # nfolds ideally took more time but I believe it is the correct
                                             # way of doing rather than taking validation set.
                                            ),
                         hyper_params=hyper_params,
                         search_criteria=search_crit,
                         grid_id='g2') # name of the grid

XGB_grid.train(x=vars_to_use,
               y='target',
               training_frame=h2o_df_train) 

In [89]:
# exploring the modesl 
XGB_grid.get_grid()

    learn_rate max_depth min_rows ntrees sample_rate   model_ids  accuracy
0        0.005         8     10.0   1360         0.8  g2_model_1  0.782824
1        0.001         8     20.0     46         0.6  g2_model_2  0.773228


In [122]:
# Best model
best_XG_Grid 

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  g2_model_1


ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.13601975447423742
RMSE: 0.3688085607388167
LogLoss: 0.4166825112634517
Mean Per-Class Error: 0.2001375184104801
AUC: 0.8892462043527933
pr_auc: 0.8805401159906923
Gini: 0.7784924087055867
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.41581707225976355: 


,0,1,Error,Rate
0,90664.0,36505.0,0.2871,(36505.0/127169.0)
1,15199.0,107632.0,0.1237,(15199.0/122831.0)
Total,105863.0,144137.0,0.2068,(51704.0/250000.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4158171,0.8063288,232.0
max f2,0.1889161,0.8824624,316.0
max f0point5,0.6294731,0.8092707,152.0
max accuracy,0.5087246,0.799644,198.0
max precision,0.9910524,1.0,0.0
max recall,0.0077728,1.0,395.0
max specificity,0.9910524,1.0,0.0
max absolute_mcc,0.4889501,0.5998570,205.0
max min_per_class_accuracy,0.5087246,0.7994155,198.0
max mean_per_class_accuracy,0.4889501,0.7998625,205.0


Gains/Lift Table: Avg response rate: 49.13 %, avg score: 49.13 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2020-07-11 20:16:51,1:38:02.233,0.0,0.5,0.6931472,0.5,0.0,1.0,0.508676
,2020-07-11 20:18:05,1:39:15.936,100.0,0.4349465,0.5680203,0.8718468,0.8619663,2.0117071,0.225152
,2020-07-11 20:19:15,1:40:26.730,200.0,0.4040003,0.5063546,0.8743722,0.8654553,2.0198484,0.2203
,2020-07-11 20:20:28,1:41:39.228,300.0,0.3896965,0.4736329,0.8763788,0.8677672,2.0214767,0.21852
,2020-07-11 20:21:42,1:42:53.173,400.0,0.3829362,0.4555570,0.8780196,0.8699982,2.0247332,0.218492
,2020-07-11 20:22:57,1:44:08.310,500.0,0.3794422,0.4448940,0.8792473,0.8705343,2.0271756,0.214904
,2020-07-11 20:24:14,1:45:25.659,600.0,0.3774516,0.4383575,0.8802815,0.8716622,2.0296179,0.214676
,2020-07-11 20:25:32,1:46:43.684,700.0,0.3760536,0.4338692,0.8813051,0.8713452,2.0312462,0.214808
,2020-07-11 20:26:52,1:48:02.985,800.0,0.3748880,0.4304762,0.8823938,0.8727218,2.0328744,0.21252
,2020-07-11 20:28:14,1:49:24.995,900.0,0.3738237,0.4276517,0.8834868,0.8741974,2.0336886,0.21114


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
f10,7722454.0,1.0,0.6846953
e19,871827.1875000,0.1128951,0.0772987
e18,240750.7656250,0.0311754,0.0213457
f02,227124.7812500,0.0294110,0.0201375
f27.F,186000.6406250,0.0240857,0.0164914
---,---,---,---
f09.K,8.9732685,0.0000012,0.0000008
c05.C,7.6248779,0.0000010,0.0000007
b03.N,6.3360138,0.0000008,0.0000006
b02.N,3.8477745,0.0000005,0.0000003



See the whole table with table.as_data_frame()


In [108]:
# Implementin the model to test data
grid_XGBoost = XGB_grid.get_grid(sort_by='accuracy', decreasing=True)
best_XG_Grid = grid_XGBoost.models[0]


preds = best_XG_Grid.predict(h2o_df_test)
df_test['Predicted'] = np.round(preds[2].as_data_frame(), 5)
df_preds_dt = df_test[['unique_id', 'Predicted']].copy()
df_test[['unique_id', 'Predicted']].to_csv(dirPOutput + '250k_XGBoost_Grid.csv', index=False)

# Kaggle Accuracy
# Accuracy = 0.86532
# From 0.8381

/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e20' has levels not trained on: [30146, BE271, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e13' has levels not trained on: [Q, S]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e24' has levels not trained on: [J, M, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e03' has levels not trained on: [J, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'c09' has levels not trained on: [nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'a18' has levels not trained on: [D]
  warnings.warn(w)


### HyperOpt

In [15]:
# Splitting train data into valid data rather than n-folds because of speed efficiency. Even though I would prefer n-folds, I
# wanted to make this experiements since majority of people have used it in this way
train, valid = h2o_df_train.split_frame(ratios=[.7], seed = 2020)

In [16]:
# H20 Documentation says that the dependent variable has to be in factors
train[var_dep] = train[var_dep].asfactor()
valid[var_dep]   = valid[var_dep].asfactor()

In [27]:
# Objective
def hyperopt_objective(params):
    XGB_hpropt = H2OXGBoostEstimator(col_sample_rate_per_tree = .9, # Specify the column subsampling rate per tree. It
                                    # is multiplicative with col_sample_rate, so setting both parameters to 0.8, for example,
                                    # results in 64% of columns being considered at any given node to split.
                                    # 0.9 has been taken because it seemed the most common choice among the online blogs
                                   score_tree_interval = 100, # Scoring the model after every so many trees
                                   seed = 2020, max_runtime_secs= 1800, # Each XGBoost will run for maximum 30 minutes
                                   stopping_rounds = 4,  # Stops training when the option selected for stopping_metric doesn’t
                                    # improve for the specified number of  training rounds, based on a simple moving average. 
                                   stopping_metric = "AUC", # Since the kaggle score is based upon AUC, therefore I am using AUC as metrc
                                   stopping_tolerance = 1e-3, # to stop training if the improvement is less than this value. 1e-3 is the
                                     # default value
                                   ntrees = int(params['ntrees']), # Discussed under "Hyper-parameters to optimise"
                                   max_depth = int(params['max_depth']), # Discussed under "Hyper-parameters to optimise"
                                   learn_rate = params['learn_rate'], # Discussed under "Hyper-parameters to optimise"
                                   sample_rate = params['sample_rate'], # Discussed under "Hyper-parameters to optimise"
                                   min_rows = params['min_rows'] # Discussed under "Hyper-parameters to optimise"
                                   # ,max_runtime_secs = 18000, 
                                  )
    
    
    # training the model
    XGB_hpropt.train(x=vars_to_use,
               y='target',
               training_frame=train)
    

    # validating the model on validation frame
    preds = XGB_hpropt.predict(valid)
    preds = preds[2].as_data_frame()['p1'].values
    score = roc_auc_score(valid[var_dep].as_data_frame().as_matrix(), preds)
    
    return -score 

In [28]:
# Search Space
search_space = {
    'ntrees': hyperopt.hp.quniform('ntrees', 1200, 4000, 1)
    , 'max_depth': hyperopt.hp.quniform('max_depth', 4,8, 1)
    , 'learn_rate': hyperopt.hp.uniform('learn_rate', 0.001, 0.01)
    , 'sample_rate': hyperopt.hp.uniform('sample_rate', 0.4, 0.9)
    , 'min_rows': hyperopt.hp.quniform('min_rows', 10 , 40, 1)
}


In [29]:
# Optimisation
trials = hyperopt.Trials()

h2o.no_progress()
best = hyperopt.fmin(hyperopt_objective,
                     space=search_space,
                     algo=hyperopt.tpe.suggest,
                     max_evals=10, # Each model will take 30 minutes and there will be 10 models like this, thus 5 hours
                     trials=trials,
                     rstate=RandomState(2020)
)

print(best)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



 10%|█         | 1/10 [30:57<4:38:35, 1857.28s/it, best loss: -0.8810321924378307]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



 20%|██        | 2/10 [58:04<3:58:25, 1788.15s/it, best loss: -0.8811926515395831]

...

 80%|████████  | 8/10 [2:55:17<38:25, 1152.86s/it, best loss: -0.8820643226233149]  

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



 90%|█████████ | 9/10 [3:26:04<22:41, 1361.22s/it, best loss: -0.8820643226233149]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



100%|██████████| 10/10 [3:40:01<00:00, 1203.76s/it, best loss: -0.8820643226233149]
{'learn_rate': 0.00632500998161564, 'max_depth': 8.0, 'min_rows': 37.0, 'ntrees': 3785.0, 'sample_rate': 0.7140605409926624}


In [25]:
# The best model returned the value of 
#     {'learn_rate': 0.00632500998161564,
#      'max_depth': 8.0,
#      'min_rows': 37.0,
#      'ntrees': 3785.0,
#      'sample_rate': 0.7140605409926624}

# I actually did not run the model next time but by mistake run this cell, therefore got this error
best

NameError: name 'best' is not defined

In [19]:
# Implementing the best model returned by Hyperopt
XGB_hpropt_bst = H2OXGBoostEstimator(col_sample_rate_per_tree = .9,
                               score_tree_interval = 100,
                               seed = 2020, max_runtime_secs= 3600,
                               stopping_rounds = 4, 
                               stopping_metric = "AUC", 
                               stopping_tolerance = 1e-3,
                               ntrees = 3785,
                               max_depth = 8,
                               learn_rate = 0.00632500998161564,
                               sample_rate = 0.7140605409926624,
                               min_rows = 37, nfolds = 5
                               # ,max_runtime_secs = 18000, 
                              )


XGB_hpropt_bst.train(x=vars_to_use,
           y='target',
           training_frame=train)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [21]:
preds = XGB_hpropt_bst.predict(h2o_df_test)
df_test['Predicted'] = np.round(preds[2].as_data_frame(), 5)
df_preds_dt = df_test[['unique_id', 'Predicted']].copy()
df_test[['unique_id', 'Predicted']].to_csv(dirPOutput + '250k_XGBoost_hyperoptt_test_2ndver.csv', index=False)



xgboost prediction progress: |████████████████████████████████████████████| 100%


/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e20' has levels not trained on: [30146, BE271, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e13' has levels not trained on: [Q, S]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e24' has levels not trained on: [J, M, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'e03' has levels not trained on: [J, nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'f03_e11' has levels not trained on: [C_G]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'c09' has levels not trained on: [nan]
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packag

In [22]:
XGB_hpropt_bst

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1594806448431_1


ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.13771069972813313
RMSE: 0.37109392305470745
LogLoss: 0.42207939087711466
Mean Per-Class Error: 0.20218874513834284
AUC: 0.8863087173166594
pr_auc: 0.8771276603825596
Gini: 0.7726174346333188
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4171403104482695: 


,0,1,Error,Rate
0,63288.0,25722.0,0.289,(25722.0/89010.0)
1,10819.0,75329.0,0.1256,(10819.0/86148.0)
Total,74107.0,101051.0,0.2086,(36541.0/175158.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4171403,0.8048013,231.0
max f2,0.1880436,0.8816929,318.0
max f0point5,0.6272633,0.8067679,150.0
max accuracy,0.4848333,0.7974457,204.0
max precision,0.9876546,1.0,0.0
max recall,0.0062810,1.0,398.0
max specificity,0.9876546,1.0,0.0
max absolute_mcc,0.4830881,0.5959146,205.0
max min_per_class_accuracy,0.5073536,0.7962813,195.0
max mean_per_class_accuracy,0.4848333,0.7978113,204.0


Gains/Lift Table: Avg response rate: 49.18 %, avg score: 49.19 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2020-07-15 10:45:12,49 min 10.086 sec,0.0,0.5,0.6931472,0.5,0.0,1.0,0.5081698
,2020-07-15 10:46:06,50 min 4.280 sec,100.0,0.4270245,0.5526871,0.8714396,0.8616882,2.0100116,0.2247685
,2020-07-15 10:46:59,50 min 57.128 sec,200.0,0.3972490,0.4913000,0.8745307,0.8655151,2.0158142,0.2200756
,2020-07-15 10:47:54,51 min 52.382 sec,300.0,0.3856947,0.4627185,0.8764400,0.8669533,2.0192957,0.2164160
,2020-07-15 10:48:47,52 min 45.473 sec,400.0,0.3809107,0.4485616,0.8777452,0.8679981,2.0192957,0.2169641
,2020-07-15 10:49:41,53 min 38.838 sec,500.0,0.3784605,0.4406499,0.8789008,0.8697821,2.0204562,0.2157252
,2020-07-15 10:50:35,54 min 33.051 sec,600.0,0.3768100,0.4355581,0.8801161,0.8709247,2.0204562,0.2140068
,2020-07-15 10:51:30,55 min 27.952 sec,700.0,0.3755155,0.4319546,0.8813186,0.8710628,2.0204562,0.2131504
,2020-07-15 10:52:26,56 min 23.782 sec,800.0,0.3743740,0.4291463,0.8825444,0.8727993,2.0192957,0.2125053
,2020-07-15 10:53:25,57 min 23.477 sec,900.0,0.3731077,0.4263142,0.8839729,0.8745666,2.0181352,0.2113406


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
f10,4053835.7500000,1.0,0.6988023
e19,446379.3437500,0.1101128,0.0769471
e18,121911.7187500,0.0300732,0.0210152
f02,121454.7187500,0.0299604,0.0209364
f27.F,93993.9062500,0.0231864,0.0162027
---,---,---,---
e21.P,10.0393209,0.0000025,0.0000017
e21.J,8.7862110,0.0000022,0.0000015
c05.A,8.7393188,0.0000022,0.0000015
e21.N,5.7316165,0.0000014,0.0000010



See the whole table with table.as_data_frame()


##### Old model (not so important)
I ran 40 models for 2 mins each, and was not expecting a better results since each model got only a time frame of 2 mins. But to my surprise this model yielded better Kaggle score of 0.867 compare to 0.865 of the above hyperopt

In [86]:
# Optimisation - OLD
trials = hyperopt.Trials()

h2o.no_progress()
best = hyperopt.fmin(hyperopt_objective,
                     space=search_space,
                     algo=hyperopt.tpe.suggest,
                     max_evals=40,
                     trials=trials,
                     rstate=RandomState(2020)
)

print(best)

  0%|          | 0/40 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



  2%|▎         | 1/40 [02:04<1:20:39, 124.10s/it, best loss: -0.8674585083914759]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



  5%|▌         | 2/40 [04:08<1:18:42, 124.28s/it, best loss: -0.870175121415311] 

...

 95%|█████████▌| 38/40 [1:18:51<04:08, 124.42s/it, best loss: -0.8718621667871277]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



 98%|█████████▊| 39/40 [1:20:56<02:04, 124.37s/it, best loss: -0.8718621667871277]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.



100%|██████████| 40/40 [1:23:00<00:00, 124.49s/it, best loss: -0.8719438506668313]
{'learn_rate': 0.008933646880323847, 'max_depth': 8.0, 'min_rows': 10.0, 'ntrees': 2908.0, 'sample_rate': 0.5396549266120586}


In [109]:
best.

{'learn_rate': 0.008933646880323847,
 'max_depth': 8.0,
 'min_rows': 10.0,
 'ntrees': 2908.0,
 'sample_rate': 0.5396549266120586}